In [1]:
#basic setting and configurations:
!pip install -q accelerate==0.21.0
!pip install -q peft==0.4.0
!pip install -q trl==0.4.7
!pip install -q bitsandbytes==0.40.2
!pip install -q transformers==4.31.0
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

In [2]:
model_name = "NousResearch/llama-2-7b-chat-hf"
#load the dataset used for training and testing:
dataset_name = "David99YY/experiment_CoT"
new_model = "llama-2-7b-miniguanaco"

In [ ]:
!huggingface-cli login
dataset = load_dataset(dataset_name, split="train")

In [4]:
tmp = []
for i in range(0,len(dataset['input'])):
    tmp.append(('Please understand the reasoning of sentiment here with input, instruction, and output, where in the output, 0 mean negative, 1 means neutral, and 2 means positive'+
                '[/Input]'+
                dataset['input'][i] +
                '[/Instruction]' +
                dataset['instruction'][i] + '[/Answer]' + str(dataset['output'][i])))

In [5]:
df = pd.DataFrame({'input': tmp})
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())
dataset = Dataset(pa.Table.from_pandas(df))

In [7]:
b_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=getattr(torch, "float16"),
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=b_config,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=50,
    learning_rate=2e-4,
    weight_decay=0.002,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="input",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,1.709300
100,1.071900
150,1.000300
200,0.994600
250,0.996200


TrainOutput(global_step=250, training_loss=1.1544498443603515, metrics={'train_runtime': 490.8043, 'train_samples_per_second': 2.037, 'train_steps_per_second': 0.509, 'total_flos': 2315084994478080.0, 'train_loss': 1.1544498443603515, 'epoch': 1.0})